# Sample training script with naive competitive self-play.

### Switch directory in Google drive so as to import CDA env.


In [18]:
# from google.colab import drive 
# drive.mount('/content/gdrive')

# %cd "/root/ray_results/"
# !ls -l
# #!rm -rf PPO_continuousDoubleAuction-v0_*
# !ls -l
# !pwd

# %cd "/content/gdrive/My Drive/Colab Notebooks/gym-continuousDoubleAuction/"
# !ls -l

# #!pip install -r requirements.txt

# #!pip install tensorflow==2.2.0
# #!pip install ray[rllib]==0.8.5

# #!pip show tensorflow
# #!pip show ray

# #!pip freeze > requirements.txt

In [19]:
# !pip install sortedcontainers

In [20]:
# !!pip install scikit-learn

In [21]:
# !pip install tabulate

In [22]:
# !pip install -U ipywidgets

In [23]:
# !pip install matplotlib

In [24]:
# !pip install pettingzoo

In [25]:
# !pwd

In [26]:
# os.chdir('..')

In [27]:
# !pip install -e.

In [28]:
# !pip uninstall continuousDoubleAuction
# !pip uninstall continuousDoubleAuction-v0

In [29]:
# !pip show continuousDoubleAuction
# !pip show continuousDoubleAuction-v0

In [30]:
# os.chdir('gym_continuousDoubleAuction')

In [31]:
# !pwd

### Imports

In [32]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
os.environ['RAY_DEBUG_DISABLE_MEMORY_MONITOR'] = "True"

import argparse

# import gym
import gymnasium as gym

import random
import numpy as np

from collections import defaultdict
from typing import Dict

import ray
from ray import tune
from ray.rllib.utils import try_import_tf
from ray.rllib.env.wrappers.pettingzoo_env import PettingZooEnv
from ray.tune.registry import register_env
from ray.rllib.models import ModelCatalog
from ray.rllib.policy import Policy

# from ray.rllib.agents.ppo import ppo
# from ray.rllib.agents.ppo.ppo import PPOTrainer
from ray.rllib.algorithms.ppo import PPOConfig

from ray.rllib.env import BaseEnv
from ray.rllib.env.multi_agent_env import MultiAgentEnv
from ray.rllib.policy.sample_batch import SampleBatch

# from ray.rllib.evaluation import MultiAgentEpisode, RolloutWorker
# from ray.rllib.agents.callbacks import DefaultCallbacks
from ray.rllib.algorithms.callbacks import DefaultCallbacks

from ray.tune.logger import pretty_print

import sys
if "../" not in sys.path:
    sys.path.append("../")

from gym_continuousDoubleAuction.envs.continuousDoubleAuction_env import continuousDoubleAuctionEnv

# from gym_continuousDoubleAuction.train.model.model_handler import CustomModel_1
from gym_continuousDoubleAuction.train.model.model_handler import CustomLSTMRLModule

from gym_continuousDoubleAuction.train.policy.policy_handler import (
    make_RandomPolicy, 
    # gen_policy, 
    # set_agents_policies, 
    # create_train_policy_list,
    create_multi_agent_config,
    policy_mapping_fn,
    create_and_train_algorithm,
)
from gym_continuousDoubleAuction.train.weight.weight_handler import (
    get_trained_policies_name, get_max_reward_ind, cp_weight)
from gym_continuousDoubleAuction.train.storage.store_handler import storage
from gym_continuousDoubleAuction.train.callbk.callbk_handler import store_eps_hist_data
from gym_continuousDoubleAuction.train.logger.log_handler import (
    create_dir, log_g_store, load_g_store)
from gym_continuousDoubleAuction.train.plotter.plot_handler import (
    plot_storage, plot_LOB_subplot, plot_sum_ord_imb, plot_mid_prices)
from gym_continuousDoubleAuction.train.helper.helper import (
    ord_imb, sum_ord_imb, mid_price)

# tf = try_import_tf()
print(f'Imports all OK.')

Imports all OK.


### Global


### Policies

In [34]:
# CDA_env args
num_agents = 9
num_trained_agent = 3 # 
num_policies = num_agents # Each agent is using a separate policy
num_of_traders = num_agents
tape_display_length = 10 
tick_size = 1
init_cash = 1000000
max_step = 4096 # per episode, -1 in arg. (~7.2s/1000steps/iter)
is_render = False 

# RLlib config 
# train_policy_list = create_train_policy_list(num_trained_agent, "policy_")
#num_cpus = 0.25                                
num_gpus = 0.75 #0                       
num_cpus_per_worker = 0.25                                
num_gpus_per_worker = 0
num_workers = 2
num_envs_per_worker = 4
batch_mode = "complete_episodes" 
rollout_fragment_length = 128
train_batch_size = max_step
sgd_minibatch_size = 256
num_iters = 2

# log_base_dir = "/content/gdrive/My Drive/Colab Notebooks/gym-continuousDoubleAuction/results/"
log_base_dir = "results/"
log_dir = log_base_dir + "ray_results/"

# Chkpt & restore
local_dir = log_base_dir + "chkpt/"
chkpt_freq = 10
chkpt = 320
restore_path = "{}checkpoint_{}/checkpoint-{}".format(local_dir, chkpt, chkpt)
is_restore = True # True / False

# log & load 
log_g_store_dir = log_base_dir + "log_g_store/"
create_dir(log_base_dir)    
create_dir(log_g_store_dir)    

# Environment configuration
env_config = {
    "num_of_agents": 2,
    "init_cash": 10000,
    "tick_size": 1,
    "tape_display_length": 10,
    "max_step": 1000,
    "is_render": False
}

# get obs & act spaces from dummy CDA env
# single_CDA_env = continuousDoubleAuctionEnv(
#     num_of_traders, 
#     init_cash, 
#     tick_size, 
#     tape_display_length, 
#     max_step, 
#     is_render)
single_CDA_env = continuousDoubleAuctionEnv(env_config)
obs_space = single_CDA_env.get_observation_space(single_CDA_env.agents[0])
act_space = single_CDA_env.get_action_space(single_CDA_env.agents[0])
print(single_CDA_env.agents)  # Should be a non-empty list
print(single_CDA_env.get_observation_space(single_CDA_env.agents[0]))  # Should return a valid gym.Space
print(single_CDA_env.get_action_space(single_CDA_env.agents[0]))  # Should return a valid gym.Space

def env_creator(env_config):
    return continuousDoubleAuctionEnv(env_config)

# Register environment with ray.tune - this is the key fix!
tune.register_env("continuousDoubleAuction-v0", env_creator)

# register custom model (neural network)
ModelCatalog.register_custom_model("model_disc", CustomLSTMRLModule) 

ray.shutdown()
# start ray
ray.init(
    ignore_reinit_error=True,
    log_to_driver=True,
    num_cpus=2,
    dashboard_host="127.0.0.1",  # replaces webui_host
    dashboard_port=8265,          # default port; replaces webui_port
    # include_dashboard=True,        # default True
    include_dashboard=False,        # default True

)

# Global storage, a ray actor that run on it's own process & it needs to be declared after ray.init().
# g_store = storage.options(name="g_store", detached=True).remote(num_agents)
g_store = storage.options(name="g_store", lifetime="detached").remote(num_agents)

Folder creation failed or folder already exists: results/
Folder creation failed or folder already exists: results/log_g_store/
['agent_0', 'agent_1']
{'agent_0': Box(-inf, inf, (40,), float32), 'agent_1': Box(-inf, inf, (40,), float32)}
{'agent_0': Tuple(Discrete(3), Discrete(4), Box(-1.0, 1.0, (1,), float32), Box(0.0, 1.0, (1,), float32), Discrete(12)), 'agent_1': Tuple(Discrete(3), Discrete(4), Box(-1.0, 1.0, (1,), float32), Box(0.0, 1.0, (1,), float32), Discrete(12))}


2025-07-22 13:59:50,579	WARNING services.py:2152 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=4.89gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2025-07-22 13:59:51,732	INFO worker.py:1917 -- Started a local Ray instance.


(raylet) It looks like you're creating a detached actor in an anonymous namespace. In order to access this actor in the future, you will need to explicitly connect to this namespace with ray.init(namespace="c75b8f4a-49cb-4835-920e-5979783fa8be", ...)


In [35]:
# # Dictionary of policies
# policies = {"policy_{}".format(i): gen_policy(i, obs_space, act_space) for i in range(num_policies)}
# set_agents_policies(policies, obs_space, act_space, num_agents, num_trained_agent)
# policy_ids = list(policies.keys())

# def policy_mapper(agent_id):
#     """
#     Required in RLlib config.
#     """
#     for i in range(num_agents):
#         if agent_id == i:            
#             return "policy_{}".format(i)                

In [37]:
policies, policies_to_train = create_multi_agent_config(
    obs_space, act_space, num_agents, num_trained_agents=num_trained_agent)

policies: {'policy_0': (None, {'agent_0': Box(-inf, inf, (40,), float32), 'agent_1': Box(-inf, inf, (40,), float32)}, {'agent_0': Tuple(Discrete(3), Discrete(4), Box(-1.0, 1.0, (1,), float32), Box(0.0, 1.0, (1,), float32), Discrete(12)), 'agent_1': Tuple(Discrete(3), Discrete(4), Box(-1.0, 1.0, (1,), float32), Box(0.0, 1.0, (1,), float32), Discrete(12))}, {'model': {'custom_model': 'model_disc'}, 'gamma': 0.99}), 'policy_1': (None, {'agent_0': Box(-inf, inf, (40,), float32), 'agent_1': Box(-inf, inf, (40,), float32)}, {'agent_0': Tuple(Discrete(3), Discrete(4), Box(-1.0, 1.0, (1,), float32), Box(0.0, 1.0, (1,), float32), Discrete(12)), 'agent_1': Tuple(Discrete(3), Discrete(4), Box(-1.0, 1.0, (1,), float32), Box(0.0, 1.0, (1,), float32), Discrete(12))}, {'model': {'custom_model': 'model_disc'}, 'gamma': 0.99}), 'policy_2': (None, {'agent_0': Box(-inf, inf, (40,), float32), 'agent_1': Box(-inf, inf, (40,), float32)}, {'agent_0': Tuple(Discrete(3), Discrete(4), Box(-1.0, 1.0, (1,), float

### Call back

In [38]:
class MyCallbacks(DefaultCallbacks):
    # def on_episode_start(self, worker: RolloutWorker, base_env: BaseEnv,
    #                      policies: Dict[str, Policy],
    #                      episode: MultiAgentEpisode, **kwargs):
    def on_episode_start(self, *, worker, base_env, policies, episode, **kwargs):
        print(f"Episode {episode.episode_id} started.")
        """
        info["episode"] is a MultiAgentEpisode object.

        user_data dicts at 100000 items max, will auto replace old with new item at 1st index.
        hist_data dicts at 100 items max, will auto replace old with new item at 1st index.
        """
        #print("on_episode_start {}, _agent_to_policy {}".format(episode.episode_id, episode._agent_to_policy))

        prefix = "agt_"
        for i in range(num_agents):
            episode.user_data[prefix + str(i) + "_obs"] = []
            episode.user_data[prefix + str(i) + "_act"] = []
            episode.user_data[prefix + str(i) + "_reward"] = []
            episode.user_data[prefix + str(i) + "_NAV"] = []
            episode.user_data[prefix + str(i) + "_num_trades"] = []

            episode.hist_data[prefix + str(i) + "_reward"] = []
            episode.hist_data[prefix + str(i) + "_NAV"] = []
            episode.hist_data[prefix + str(i) + "_num_trades"] = []

    # def on_episode_step(self, worker: RolloutWorker, base_env: BaseEnv,
    #                     episode: MultiAgentEpisode, **kwargs):
    def on_episode_step(self, *, worker, base_env, episode, **kwargs):
        # Example: track something each step
        episode.user_data["my_metric"].append(1)        
        """
        pole_angle = abs(episode.last_observation_for()[2])
        raw_angle = abs(episode.last_raw_obs_for()[2])
        assert pole_angle == raw_angle
        episode.user_data["pole_angles"].append(pole_angle)
        """

        prefix = "agt_"
        for i in range(num_agents):
            obs = episode.last_raw_obs_for(i)
            #obs = episode.last_observation_for(i)
            act = episode.last_action_for(i)
            reward = episode.last_info_for(i).get("reward")
            NAV = episode.last_info_for(i).get("NAV")
            NAV = None if NAV is None else float(NAV)
            num_trades = episode.last_info_for(i).get("num_trades")
        
            if reward is None:      # goto next agent.
                continue

            episode.user_data[prefix + str(i) + "_obs"].append(obs)    
            episode.user_data[prefix + str(i) + "_act"].append(act)    
            episode.user_data[prefix + str(i) + "_reward"].append(reward)    
            episode.user_data[prefix + str(i) + "_NAV"].append(NAV)    
            episode.user_data[prefix + str(i) + "_num_trades"].append(num_trades)          

    # def on_episode_end(self, worker: RolloutWorker, base_env: BaseEnv,
    #                    policies: Dict[str, Policy], episode: MultiAgentEpisode,
    #                    **kwargs):
    def on_episode_end(self, *, worker, base_env, policies, episode, **kwargs):
        # Example: summarize at end
        mean_metric = sum(episode.user_data["my_metric"]) / len(episode.user_data["my_metric"])
        print(f"Episode {episode.episode_id} ended. Mean my_metric: {mean_metric}")        
        #print("on_episode_end {}, episode.agent_rewards {}".format(episode.episode_id, episode.agent_rewards))
        """
        arg: {"env": .., "episode": ...}
        """

        g_store = ray.util.get_actor("g_store")
        prefix = "agt_"
        for agt_id in range(num_agents):
            obs_key = prefix + str(agt_id) + "_obs"
            act_key = prefix + str(agt_id) + "_act"
            reward_key = prefix + str(agt_id) + "_reward"
            NAV_key = prefix + str(agt_id) + "_NAV"
            num_trades_key = prefix + str(agt_id) + "_num_trades"      

            # store into episode.hist_data
            store_eps_hist_data(episode, reward_key)
            store_eps_hist_data(episode, NAV_key)
            store_eps_hist_data(episode, num_trades_key)

            # store step data
            obs = episode.user_data[obs_key]
            act = episode.user_data[act_key]
            reward = episode.user_data[reward_key]
            NAV = episode.user_data[NAV_key]
            num_trades = episode.user_data[num_trades_key]
            ray.get(g_store.store_agt_step.remote(agt_id, obs, act, reward, NAV, num_trades))       
        
            # Store episode data.   
            eps_reward = np.sum(reward)
            eps_NAV = np.sum(NAV)
            eps_num_trades = np.sum(num_trades)
            ray.get(g_store.store_agt_eps.remote(agt_id, eps_reward, eps_NAV, eps_num_trades))

        ray.get(g_store.inc_eps_counter.remote())  

    # def on_sample_end(self, worker: RolloutWorker, samples: SampleBatch,
    #                   **kwargs):
    #     """
    #     arg: {"samples": .., "worker": ...}

    #     Notes:
    #         https://github.com/ray-project/ray/blob/master/rllib/policy/sample_batch.py        
    #     """
    #     #print("on_sample_end returned sample batch of size {}".format(samples.count))

    #     """
    #     MultiAgentBatch_obj = info["samples"]
    #     MultiAgentBatch_policy_batches = MultiAgentBatch_obj.policy_batches
    #     MultiAgentBatch_count = MultiAgentBatch_obj.count

    #     access_sample_batches(MultiAgentBatch_policy_batches)
    #     print("info['samples'].policy_batches = {}".format(info["samples"].policy_batches))
    #     print("info['worker'] = {}".format(info["worker"])) # RolloutWorker object
    #     """

    def on_train_result(self, trainer, result: dict, **kwargs):
        """
        info["trainer"] is the trainer object.

        info["result"] contains a bunch of info such as episodic rewards for 
        each policy in info["result"][hist_stats] dictionary.
        """
        #print("trainer.train() result: {} -> {} episodes".format(trainer, result["episodes_this_iter"]))
        # you can mutate the result dict to add new fields to return
        result["callback_ok"] = True
        #print("on_train_result result", result)
    
        train_policies_name = get_trained_policies_name(policies, num_trained_agent)    
        max_reward_ind = get_max_reward_ind(result, train_policies_name)
        max_reward_policy_name = train_policies_name[max_reward_ind]
        cp_weight(trainer, train_policies_name, max_reward_policy_name)    

        g_store = ray.util.get_actor("g_store")      
        prefix = "policy_policy_"
        suffix = "_reward"
        hist_stats = result["hist_stats"]
        eps_this_iter = result["episodes_this_iter"]
        for agt_id in range(num_agents):
            key = prefix + str(agt_id) + suffix
            for i in range(eps_this_iter):
                ray.get(g_store.store_agt_train.remote(agt_id, hist_stats[key][i]))

        #print("on_train_result info['result'] {}".format(info["result"]))     

### RLlib config

In [39]:
# def get_config():
#     config = ppo.DEFAULT_CONFIG.copy()
#     config["multiagent"] = {"policies_to_train": train_policy_list,
#                             "policies": policies,
#                             "policy_mapping_fn": policy_mapper,
#                            }    
#     #config["num_cpus"] = num_cpus     # trainer, applicable only when using tune.
#     config["num_gpus"] = num_gpus     # trainer
#     config["num_cpus_per_worker"] = num_cpus_per_worker                                
#     config["num_gpus_per_worker"] = num_gpus_per_worker                      
#     config["num_workers"] = num_workers
#     config["num_envs_per_worker"] = num_envs_per_worker  
#     config["batch_mode"] = batch_mode       # "complete_episodes" / "truncate_episodes"
#     config["train_batch_size"] = train_batch_size # Training batch size, if applicable. Should be >= rollout_fragment_length.
#                                                   # Samples batches will be concatenated together to a batch of this size,
#                                                   # which is then passed to SGD.
#     config["rollout_fragment_length"] = rollout_fragment_length # replaced "sample_batch_size",
#     config["sgd_minibatch_size"] = sgd_minibatch_size 
#     config["log_level"] = "WARN" # WARN/INFO/DEBUG 
#     config["callbacks"] = MyCallbacks
#     config["output"] = log_dir

#     return config

In [41]:
from ray.rllib.algorithms.algorithm_config import AlgorithmConfig

def get_config():
    config = (
        PPOConfig()            
        .environment(
            "continuousDoubleAuction-v0",
            # continuousDoubleAuctionEnv,
            # env_config={
            #     "num_of_agents": num_of_traders, 
            #     "init_cash": init_cash, 
            #     "tick_size": tick_size, 
            #     "tape_display_length": tape_display_length,
            #     "max_step": max_step - 1, 
            #     "is_render": is_render
            # }
            # env_config={"disable_env_checker": True},            
        ) 
        .multi_agent(
            policies=policies,
            
            # policy_mapping_fn=policy_mapper,
            policy_mapping_fn=policy_mapping_fn,
                
            # policies_to_train=train_policy_list,
            policies_to_train=policies_to_train,            
        )
        .env_runners(
            # num_env_runners=num_workers,
            num_env_runners=0,
            
            num_envs_per_env_runner=num_envs_per_worker,
            rollout_fragment_length=rollout_fragment_length,
            batch_mode=batch_mode,
        )
        .learners(
            num_learners=1,  # Typically 1 learner unless using distributed training
            num_gpus_per_learner=num_gpus,  # Trainer GPU allocation
            num_cpus_per_learner=num_cpus_per_worker,
        )
        .training(
            train_batch_size_per_learner=train_batch_size,
            # sgd_minibatch_size,
        )
        .callbacks(MyCallbacks)
        # .output_dir(log_dir)
        .framework("torch")  # Explicitly set framework if needed
    )

    # Optional: Configure resources more granularly if needed
    if num_gpus_per_worker > 0:
        config.env_runners(
            num_gpus_per_env_runner=num_gpus_per_worker
        )

    return config

### Train


In [42]:
# def go_train(config):    
#     trainer = ppo.PPOTrainer(config=config, env="continuousDoubleAuction-v0")
    
#     if is_restore == True:
#         trainer.restore(restore_path) 

#     g_store = ray.util.get_actor("g_store")          
#     result = None
#     for i in range(num_iters):
#         result = trainer.train()       
#         print(pretty_print(result)) # includes result["custom_metrics"]
#         print("training loop = {} of {}".format(i + 1, num_iters))            
#         print("eps sampled so far {}".format(ray.get(g_store.get_eps_counter.remote())))

#         if i % chkpt_freq == 0:
#             checkpoint = trainer.save(local_dir)
#             print("checkpoint saved at", checkpoint)
    
#     checkpoint = trainer.save(local_dir)
#     print("checkpoint saved at", checkpoint)

#     print("result['experiment_id']", result["experiment_id"])
    
#     return result["experiment_id"]
    
# # run everything
# experiment_id = go_train(get_config())            

In [43]:
def go_train(config):    
    # trainer = ppo.PPOTrainer(config=config, env="continuousDoubleAuction-v0")      
    # algo = config.build()   
    algo = config.build()
    
    # if is_restore == True:
    #     trainer.restore(restore_path) 

    # g_store = ray.util.get_actor("g_store")          
    # result = None
    # for i in range(num_iters):
    #     result = algo.train()       

    #     print(pretty_print(result)) # includes result["custom_metrics"]
    #     print("training loop = {} of {}".format(i + 1, num_iters))            
    #     print("eps sampled so far {}".format(ray.get(g_store.get_eps_counter.remote())))

    #     if i % chkpt_freq == 0:
    #         checkpoint = algo.save(local_dir)
    #         print("checkpoint saved at", checkpoint)
    
    # checkpoint = algo.save(local_dir)
    # print("checkpoint saved at", checkpoint)
    # print("result['experiment_id']", result["experiment_id"])

    # return result["experiment_id"]
    return None
    
# run everything
experiment_id = go_train(get_config())            

2025-07-22 14:00:46,026	WARNING deprecation.py:50 -- DeprecationWarning: `build` has been deprecated. Use `AlgorithmConfig.build_algo` instead. This will raise an error in the future!
2025-07-22 14:00:46,028	WARNING algorithm_config.py:5014 -- You are running PPO on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
/usr/local/lib/python3.10/dist-packages/ray/rllib/algorithms/algorithm.py:520: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/usr/local/lib/python3.10/dist-package

ValueError: No default encoder config for obs space={'agent_0': Box(-inf, inf, (40,), float32), 'agent_1': Box(-inf, inf, (40,), float32)}, lstm=False found.

### Plot all steps.

Agt_0, 1, 2 are trained agents (with PPO) while the rest are random agents.

In [ ]:
# plot_storage(num_agents, init_cash)

In [ ]:
# plot_storage(num_agents, init_cash, "step", "NAV")

In [ ]:
# plot_storage(num_agents, init_cash, "step", "num_trades")

### Log/load last episode

In [ ]:
# log_g_store(log_g_store_dir, num_agents, experiment_id)
# load_g_store(log_g_store_dir, num_agents, experiment_id)

### Plot steps from last episode.

In [ ]:
# plot_storage(num_agents, init_cash)

In [ ]:
# plot_storage(num_agents, init_cash, "step", "NAV")

In [ ]:
# plot_storage(num_agents, init_cash, "step", "num_trades")

### LOB from last episode.

In [ ]:
# g_store = ray.util.get_actor("g_store")          
# #store = ray.get(g_rere.get_storage.remote())

# depth = 10
# bid_size, bid_price, ask_size, ask_price = ray.get(g_store.get_obs_from_agt.remote(0, depth))

### LOB order imbalance

In [ ]:
# ord_imb_store = ord_imb(bid_size, ask_size)
# plot_LOB_subplot(ord_imb_store, depth, '_ord_imb') 

### LOB sum of order imbalance

In [ ]:
# ord_imb_store = np.asarray(ord_imb_store)
# sum_ord_imb_store = sum_ord_imb(ord_imb_store)
# plot_sum_ord_imb(sum_ord_imb_store, "sum_ord_imb")

### LOB mid price (subplot)

In [ ]:
# mid_price_store = mid_price(bid_price, ask_price)
# plot_LOB_subplot(mid_price_store, depth, '_mid_price')    

### LOB mid prices 

In [ ]:
# plot_mid_prices(mid_price_store,)

### LOB bid size

In [ ]:
# plot_LOB_subplot(bid_size, depth, '_bid_size')    

### LOB ask size

In [ ]:
# plot_LOB_subplot(ask_size, depth, '_ask_size')    

### LOB bid price

In [ ]:
# plot_LOB_subplot(bid_price, depth, '_bid_price')    

### LOB ask price

In [ ]:
# plot_LOB_subplot(ask_price, depth, '_ask_price')    

In [ ]:
# from ray.rllib.env.multi_agent_env import MultiAgentEnv
# from ray.rllib.algorithms.ppo import PPOConfig
# from ray.rllib.policy.policy import PolicySpec
# from ray import tune  # Important: use ray.tune, not ray.tune.registry
# from gymnasium import spaces
# import numpy as np
# import ray

# class CustomMARLEnv(MultiAgentEnv):
#     # def __init__(self, config=None):
#     def __init__(self, p1):    
#         super().__init__()
#         self.agents = ["agent_0", "agent_1", "agent_2"]
#         self._agent_ids = set(self.agents)
        
#         # Define spaces as dictionaries mapping agent_id to space
#         obs_space = spaces.Box(low=-1, high=1, shape=(4,), dtype=np.float32)
#         act_space = spaces.Discrete(3)
        
#         self.observation_space = {agent: obs_space for agent in self.agents}
#         self.action_space = {agent: act_space for agent in self.agents}
        
#     def reset(self, *, seed=None, options=None):
#         observations = {agent: self.observation_space[agent].sample() for agent in self.agents}
#         infos = {agent: {} for agent in self.agents}
#         return observations, infos
    
#     def step(self, action_dict):
#         observations = {agent: self.observation_space[agent].sample() for agent in self.agents}
#         rewards = {agent: np.random.random() for agent in self.agents}
        
#         terminateds = {agent: False for agent in self.agents}
#         truncateds = {agent: False for agent in self.agents}
#         # Global episode termination (required!)
#         terminateds["__all__"] = False  # Set to True when episode should end
#         truncateds["__all__"] = False   # Set to True when episode should be truncated        
        
#         infos = {agent: {} for agent in self.agents}
        
#         return observations, rewards, terminateds, truncateds, infos

# def env_creator(env_config):
#     return CustomMARLEnv(env_config)

# # Initialize Ray first
# ray.shutdown()
# ray.init(ignore_reinit_error=True)

# # Register environment with ray.tune - this is the key fix!
# tune.register_env("custom_marl_env", env_creator)

# # Create config with correct new API
# config = (
#     PPOConfig()
#     .environment("custom_marl_env")
#     .multi_agent(
#         policies={
#             "shared_policy": PolicySpec(),
#         },
#         policy_mapping_fn=lambda agent_id, episode, **kwargs: "shared_policy",
#     )
#     .env_runners(
#         num_env_runners=2,
#     )
#     .training(
#         train_batch_size=1000,
#         num_sgd_iter=10,
#     )
# )

# # Build and train
# try:
#     algo = config.build()
    
#     for i in range(3):
#         result = algo.train()
#         # Try different possible result keys
#         reward_key = None
#         for key in ['env_runners/episode_reward_mean', 'episode_reward_mean', 'hist_stats/episode_reward']:
#             if key in result:
#                 reward_key = key
#                 break
        
#         if reward_key:
#             print(f"Iteration {i}: reward = {result[reward_key]}")
#         else:
#             print(f"Iteration {i}: training completed")
    
#     algo.stop()
    
# except Exception as e:
#     print(f"Error: {e}")
#     if 'algo' in locals():
#         algo.stop()

# ray.shutdown()

In [ ]:
from ray.rllib.env.multi_agent_env import MultiAgentEnv
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.policy.policy import PolicySpec
from ray import tune  # Important: use ray.tune, not ray.tune.registry
from gymnasium import spaces
import numpy as np
import ray

class CustomMARLEnv(MultiAgentEnv):
    def __init__(self, config=None):
        super().__init__()
        
        # Extract parameters from config with defaults
        config = config or {}
        self._num_agents = config.get("num_agents", 3)  # Use underscore to avoid conflicts
        self.obs_dim = config.get("obs_dim", 4)
        self.num_actions = config.get("num_actions", 3)
        self.max_steps = config.get("max_steps", 100)
        
        # Initialize agents based on parameters
        self.agents = [f"agent_{i}" for i in range(self._num_agents)]
        self._agent_ids = set(self.agents)
        self.current_step = 0
        
        # Define spaces as dictionaries mapping agent_id to space
        obs_space = spaces.Box(low=-1, high=1, shape=(self.obs_dim,), dtype=np.float32)
        act_space = spaces.Discrete(self.num_actions)
        
        self.observation_space = {agent: obs_space for agent in self.agents}
        self.action_space = {agent: act_space for agent in self.agents}
        
        print(f"Initialized environment with {len(self.agents)} agents: {self.agents}")
    
    def get_agent_ids(self):
        """Return the set of agent IDs."""
        return self._agent_ids
        
    def reset(self, *, seed=None, options=None):
        self.current_step = 0
        observations = {agent: self.observation_space[agent].sample() for agent in self.agents}
        infos = {agent: {} for agent in self.agents}
        return observations, infos
    
    def step(self, action_dict):
        self.current_step += 1
        
        # Validate that we received actions for active agents
        if not action_dict:
            raise ValueError("No actions received")
        
        print(f"Step {self.current_step}: Received actions for {list(action_dict.keys())}")
        print(f"Expected agents: {self.agents}")
        
        # Only generate observations for agents that are still active
        active_agents = [agent for agent in self.agents if agent in action_dict]
        
        observations = {}
        rewards = {}
        terminateds = {}
        truncateds = {}
        infos = {}
        
        for agent in active_agents:
            observations[agent] = self.observation_space[agent].sample()
            rewards[agent] = np.random.random()
            terminateds[agent] = False
            truncateds[agent] = False
            infos[agent] = {}
        
        # Global episode termination (required!)
        terminateds["__all__"] = False
        truncateds["__all__"] = self.current_step >= self.max_steps
        
        return observations, rewards, terminateds, truncateds, infos

def env_creator(env_config):
    print(f"Creating environment with config: {env_config}")
    env = CustomMARLEnv(env_config)
    print(f"Environment created successfully with agents: {env.agents}")
    return env

# Initialize Ray first
ray.shutdown()
ray.init(ignore_reinit_error=True)

# Register environment with ray.tune - this is the key fix!
tune.register_env("custom_marl_env", env_creator)

# Create config with environment parameters
env_config = {
    "num_agents": 3,  # Start with fewer agents for debugging
    "obs_dim": 4,
    "num_actions": 3,
    "max_steps": 50,  # Shorter episodes for testing
}

print(f"Using env_config: {env_config}")

config = (
    PPOConfig()
    .environment(
        "custom_marl_env",
        env_config=env_config  # Pass parameters to your environment
    )
    .multi_agent(
        policies={
            "shared_policy": PolicySpec(),
        },
        policy_mapping_fn=lambda agent_id, episode, **kwargs: "shared_policy",
    )
    .env_runners(
        num_env_runners=1,  # Start with 1 for debugging
    )
    .training(
        train_batch_size=100,  # Smaller batch for testing
        # sgd_minibatch_size=64,
        num_sgd_iter=5,
    )
    .debugging(log_level="DEBUG")  # Add debugging
)

# Build and train
try:
    algo = config.build()
    
    for i in range(10):
        result = algo.train()
        # Try different possible result keys
        reward_key = None
        for key in ['env_runners/episode_reward_mean', 'episode_reward_mean', 'hist_stats/episode_reward']:
            if key in result:
                reward_key = key
                break
        
        if reward_key:
            print(f"Iteration {i}: reward = {result[reward_key]}")
        else:
            print(f"Iteration {i}: training completed")
    
    algo.stop()
    
except Exception as e:
    print(f"Error: {e}")
    if 'algo' in locals():
        algo.stop()

ray.shutdown()